<a href="https://colab.research.google.com/github/ShykerBogdan/Prozzoro_project/blob/master/DataPrepr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import time
from datetime import datetime

In [0]:
filename='/content/gdrive/My Drive/ML_proz/data_source.csv'
df=pd.read_csv(filename)
df.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5,14,15,16,17,18,20,24,38,48,50,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3507778, 56)

In [0]:
freq = df['participants'].value_counts()

def count_lots(raw, frequency):
  raw['count_lots'] = frequency[raw['participants']]
  return raw

feature_data = df.apply(count_lots, args=(freq,), axis=1)


In [0]:



class DataPreproccessing:
    def __init__(self, data_source, churn_interval, start_period=None, end_period=None):
        self.last_day=pd.to_datetime('2018-11-29')
        # для статистики постачальника вибираємо проміжок [start_period,end_period]
        self.divide_data = pd.to_datetime('2018-12-20') - pd.DateOffset(days=churn_interval)
        print(self.divide_data)
        self.data_source = data_source[pd.to_datetime(data_source['date_from_id']) <= self.divide_data]
        # для labeling вибираємо період [end_period, end_period+churn_interval]
        self.label_data = data_source[pd.to_datetime(data_source['date_from_id']) > self.divide_data]
        # вибираємо унікальних постачальників
        self.unique_id = list(self.data_source['participants'].value_counts().index)
        # створюємо dataframe з колонкою унікальних постачальників і туди будемо додавати features
        self.feature_data = pd.DataFrame(self.unique_id, columns=['unique_id'])
     
    
    def get_label(self):
        participants=list(self.label_data['participants'].value_counts().index)
        print(len(participants))
        def get_y(raw,participants):
          if raw['unique_id'] in participants:
            raw['y']=1
          else:
            raw['y']=0
          return raw
        
        self.feature_data=self.feature_data.apply(get_y,args=(participants,),axis=1)
        
    def create_features(self):
        self.get_label()
        self.data_source['winner']=self.data_source[['winner','contracts.status']].apply(lambda x: x['winner'] if pd.isnull(x['contracts.status']) else
                                                                                         (0 if x['contracts.status']=='cancelled' else x['winner'] ),axis=1)
        self.activity_date()
#         ---------------
        self.count_lots()
        print('count_lots has already finish')
        self.count_win_lose()
        print('count_win_lose has already finish')
        self.count_win_type_procedure()
        print('count_win_type_procedure has already finish')
        self.count_lose_type_procedure()
        print('count_lose_type_procedure has already finish')
        self.feature_data['count_lots_open']=self.feature_data['win_open']+self.feature_data['lose_open']
        self.feature_data['count_lots_not_open']=self.feature_data['win_not_open']+self.feature_data['lose_not_open']
        print('count_lots_open has already finish')
        self.last_activity()
    def count_lots(self):
        freq = self.data_source['participants'].value_counts()

        def count_lots(raw, frequency):
            raw['count_lots'] = frequency[raw['unique_id']]
            return raw

        self.feature_data = self.feature_data.apply(count_lots, args=(freq,), axis=1)

    def count_win_lose(self):
        data=self.data_source[self.data_source['winner']==1]
        frequency=data['participants'].value_counts()
        
        def win_lose(raw,frequency):
            # use try bsc if index not in frequency 'win'=0
            try:
              raw['win']=frequency[raw['unique_id']]
            except:
              raw['win']=0
            return raw
            

        self.feature_data = self.feature_data.apply(win_lose, args=(
        frequency,),axis=1)
        self.feature_data['lose']=self.feature_data['count_lots']-self.feature_data['win']
        
    # всьо в одну функцію
    def count_win_type_procedure(self):
        data=self.data_source[self.data_source['winner']==1]
        data=data[data['tenders.procurementMethod']=='open']
        frequency=data['participants'].value_counts()
        def count_win_open(raw,frequency):
            try:
              raw['win_open']=frequency[raw['unique_id']]
            except:
              raw['win_open']=0
            return raw
              
        self.feature_data = self.feature_data.apply(count_win_open, args=(
        frequency,),axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))
        self.feature_data['win_not_open']=self.feature_data['win']-self.feature_data['win_open']
    
    
    def count_lose_type_procedure(self):
        data=self.data_source[self.data_source['winner']==0]
        data=data[data['tenders.procurementMethod']=='open']
        frequency=data['participants'].value_counts()
        def count_lose_open(raw,frequency):
            try:
              raw['lose_open']=frequency[raw['unique_id']]
            except:
              raw['lose_open']=0
            return raw
              
        self.feature_data = self.feature_data.apply(count_lose_open, args=(
        frequency,),axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))
        self.feature_data['lose_not_open']=self.feature_data['lose']-self.feature_data['lose_open']
    
    
    def activity_date(self):
        f=['winner','tenders.auctionPeriod_startDate','contracts.dateSigned','date_from_id','awards.date']
        self.data_source['new_date']=self.data_source[f].apply(lambda x:x[f[2]] if x[f[0]==1] else (x[f[3]] if pd.isnull(x[f[1]]) else x[f[1]] ),axis=1)
        self.data_source['new_date']=self.data_source[['new_date']].apply(lambda x: x['new_date'][:10],axis=1)
        self.data_source['new_date']=self.data_source[['new_date','date_from_id']].apply(lambda x: x['date_from_id'] if x['new_date']=='\\N' else x['new_date'],axis=1)
        
    def last_activity(self):
        data=self.data_source[['participants','new_date']]
        data['new_date'] =  pd.to_datetime(data['new_date'], format='%Y/%m/%d')
        data=data.sort_values(by='new_date')
        self.divide_data=str(self.divide_data)[:10]
        
        def get_last_activity(raw,data):
          raw['last_activity_date']=str(data[data['participants']==raw['unique_id']].iloc[-1]['new_date'])[:10]
          raw['last_activity_days']=(pd.to_datetime(self.divide_data)-pd.to_datetime(raw['last_activity_date'])).days
          return raw
        
        
        self.feature_data=self.feature_data.apply(get_last_activity,args=(data,),axis=1)
        self.feature_data['last_activity_days']=self.feature_data.apply(lambda x: 0 if x['last_activity_days']<0 else x['last_activity_days'],axis=1)
          
          
start_time = time.time()
prepr = DataPreproccessing(df.iloc[:1000], 5)
prepr.create_features()
d = prepr.feature_data
print(d['y'].value_counts())
print(d['lots_count'].value_counts())
print("--- %s seconds ---" % (time.time() - start_time))
# 158 sec - 30000

2017-08-07 00:00:00
581
count_lots has already finish
count_win_lose has already finish
--- 1.2714004516601562 seconds ---
count_win_type_procedure has already finish
--- 1.5596635341644287 seconds ---
count_lose_type_procedure has already finish
count_lots_open has already finish


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    355
1     19
Name: y, dtype: int64
--- 2.668469190597534 seconds ---
